In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import json
import pandas as pd
from geojson import Feature, FeatureCollection, Point
import tabula
import re

In [2]:
# Gotta figure out how to get rid of the java error.
# SO: https://stackoverflow.com/questions/54817211/java-command-is-not-found-from-this-python-process-please-ensure-java-is-inst

dsf = tabula.read_pdf('condo.pdf', pages='all',lattice=True)

In [3]:
df = dsf[0]
df.columns = df.iloc[0]
df = df.drop(df.index[0])
### This next line gets ride of the Nan ###
df = df.iloc[: , 1:]
df = df.replace(np.nan, 'Not Available', regex=True)
df = df.rename(columns={'Building / Address / City': 'building_address_city','Days on\rMarket':'days_on_market',\
                        'Price /\rSq. Ft.':'price_per_sqft'})
df['geo_address'] = df['building_address_city']
df['geo_address'] = df['geo_address'].map(lambda x: re.sub(r'\r', ' ', x))
df = df.replace('\n',' ',regex=True)
df2 = df['geo_address'].str.extract(r'(?P<building_name>\D*)(?P<address_only>\d.*)', expand=True)

Review this later (it's about how to properly merge DFs): https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html

In [4]:
result = pd.merge(df,df2,left_index=True, right_index=True)
df = result

In [5]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [6]:
df['loc'] = df['address_only'].apply(geolocator.geocode, timeout=10)

In [7]:
df['loc']

1     (1300 Monad Terrace, Miami Beach, FL 33139, US...
2     (102 24th St, Miami Beach, FL 33139, USA, (25....
3     (16901 Collins Ave, Sunny Isles Beach, FL 3316...
4     (102 24th St, Miami Beach, FL 33139, USA, (25....
5     (350 Ocean Dr #1105n, Key Biscayne, FL 33149, ...
6     (5875 Collins Ave, Miami Beach, FL 33140, USA,...
7     (6000 Island Blvd, Aventura, FL 33160, USA, (2...
8     (9 Island Ave APT T2, Miami Beach, FL 33139, U...
9     (520 West Ave, Miami Beach, FL 33139, USA, (25...
10    (20165 NE 39th Pl, Aventura, FL 33180, USA, (2...
Name: loc, dtype: object

In [8]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

Columns must be same length as key explained
- https://cumsum.wordpress.com/2021/06/01/pandas-valueerror-columns-must-be-same-length-as-key/

Creating `df2` based on `df` row indicies.

https://datascience.stackexchange.com/questions/77033/creating-a-new-dataframe-with-specific-row-numbers-from-another

What's tripping this error is when the lat/lon/alititude generator is fed a Nan value to parse - it's effectively trying to split a `null` value into three separate parts - which is impossible.

The code below effectively caputres the anatomy of this error. It creates a new `df` based on a list of indicies from an existing one. Row `56` has no `point` since the `geocoder` couldn't find coordinates for it. The other three do. Splitting the `point` column fails only when row `56` - with `null` values - is attempted.
```
df2 = df.iloc[[56,57,58,59],:]
df2[['lat','lon','altitude']] = pd.DataFrame(df2['point'].to_list(),index=df2.index)
```

In [9]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

df

,building_address_city,List Date,Close Date,days_on_market,Sale Price,Sq. Ft.,price_per_sqft,Agent,Buyer Agent,geo_address,building_name,address_only,loc,point,lat,lon,altitude
1,Monad Terrace\r1300 Monad Ter PH-F\rMiami Beach,9/29/2021,1/3/2022,65,"$9,900,000.00",7190,"$3,526.90",Dina Goldentayer,Jaime Blatt,Monad Terrace 1300 Monad Ter PH-F Miami Beach,Monad Terrace,1300 Monad Ter PH-F Miami Beach,"(1300 Monad Terrace, Miami Beach, FL 33139, US...","(25.7840774, -80.1425416, 0.0)",25.784077,-80.142542,0.0
2,1 Hotel & Homes\r100 24th St 1044\rMiami Beach,3/1/2021,1/7/2022,311,"$4,200,000.00",1687,"$2,489.63",Patty Marun,Priscilla Haisley,1 Hotel & Homes 100 24th St 1044 Miami Beach,,1 Hotel & Homes 100 24th St 1044 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7989302, -80.126869, 0.0)",25.798930,-80.126869,0.0
3,Jade Signature\r16901 Collins Ave 3105\rSunny ...,2/3/2021,1/6/2022,226,"$3,950,000.00",2937,"$1,344.91",Rita Collins,Yosef Lustgarten,Jade Signature 16901 Collins Ave 3105 Sunny Is...,Jade Signature,16901 Collins Ave 3105 Sunny Isles Beach,"(16901 Collins Ave, Sunny Isles Beach, FL 3316...","(25.932573, -80.1211962, 0.0)",25.932573,-80.121196,0.0
4,1 Hotel & Homes\r100 24th St 1107\rMiami Beach,1/4/2022,1/4/2022,1,"$3,775,000.00",1675,"$2,253.73",Ander Egurrola,Priscilla Haisley,1 Hotel & Homes 100 24th St 1107 Miami Beach,,1 Hotel & Homes 100 24th St 1107 Miami Beach,"(102 24th St, Miami Beach, FL 33139, USA, (25....","(25.7989302, -80.126869, 0.0)",25.798930,-80.126869,0.0
5,Oceana Key Biscayne Condo\r350 Ocean Dr 1105N\...,7/23/2021,1/5/2022,138,"$3,130,000.00",1727,"$1,812.39",Jacqueline Molina,Jacqueline Molina,Oceana Key Biscayne Condo 350 Ocean Dr 1105N K...,Oceana Key Biscayne Condo,350 Ocean Dr 1105N Key Biscayne,"(350 Ocean Dr #1105n, Key Biscayne, FL 33149, ...","(25.694538, -80.15759829999999, 0.0)",25.694538,-80.157598,0.0
6,Mei Condo\r5875 Collins Ave 1602\rMiami Beach,9/25/2021,1/5/2022,102,"$3,000,000.00",1725,"$1,739.13",Valentina Georgescu,Valentina Georgescu,Mei Condo 5875 Collins Ave 1602 Miami Beach,Mei Condo,5875 Collins Ave 1602 Miami Beach,"(5875 Collins Ave, Miami Beach, FL 33140, USA,...","(25.8408333, -80.1205556, 0.0)",25.840833,-80.120556,0.0
7,BellaMare @ Williams Island\r6000 Island Blvd ...,9/2/2020,1/5/2022,403,"$2,300,000.00",4603,$574.28,Richard Goihman,Richard Goihman,BellaMare @ Williams Island 6000 Island Blvd 1...,BellaMare @ Williams Island,6000 Island Blvd 1208 Aventura,"(6000 Island Blvd, Aventura, FL 33160, USA, (2...","(25.9427325, -80.1352515, 0.0)",25.942733,-80.135251,0.0
8,Nine Island Avenue Condo\r9 Island Ave T2\rMia...,11/14/2021,1/5/2022,30,"$2,300,000.00",2079,"$1,106.30",Nancy Batchelor PA,Mikael Hamaoui,Nine Island Avenue Condo 9 Island Ave T2 Miami...,Nine Island Avenue Condo,9 Island Ave T2 Miami Beach,"(9 Island Ave APT T2, Miami Beach, FL 33139, U...","(25.7900057, -80.1487379, 0.0)",25.790006,-80.148738,0.0
9,The Bentley Bay\r520 West Ave 2201\rMiami Beach,9/16/2021,1/6/2022,49,"$2,235,000.00",1994,"$1,120.86",Maria Teresa Andreychuk,Jeri Jenkins,The Bentley Bay 520 West Ave 2201 Miami Beach,The Bentley Bay,520 West Ave 2201 Miami Beach,"(520 West Ave, Miami Beach, FL 33139, USA, (25...","(25.7752309, -80.1419425, 0.0)",25.775231,-80.141942,0.0
10,Porto Vita\r20165 NE 39th Pl 304\rAventura,12/3/2021,1/4/2022,1,"$2,150,000.00",4320,$497.69,Diane Lieberman,Patricia Genis,Porto Vita 20165 NE 39th Pl 304 Aventura,Porto Vita,20165 NE 39th Pl 304 Aventura,"(20165 NE 39th Pl, Aventura, FL 33180, USA, (2...","(25.9624882, -80.1246262, 0.0)",25.962488,-80.124626,0.0


Folium documentation link: https://python-visualization.github.io/folium/index.html

Folium is the library that lets us use leaflet with Python, since it's meant to be used with JavaScript.

For adding titles: https://stackoverflow.com/questions/61928013/adding-a-title-or-text-to-a-folium-map

In [10]:
df.columns

Index(['building_address_city', 'List Date', 'Close Date', 'days_on_market',
       'Sale Price', 'Sq. Ft.', 'price_per_sqft', 'Agent', 'Buyer Agent',
       'geo_address', 'building_name', 'address_only', 'loc', 'point', 'lat',
       'lon', 'altitude'],
      dtype='object')

In [14]:
import folium
from folium.plugins import MarkerCluster

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=11)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent Miami-Dade Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format('January 2nd - January 10th')

marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["Sale Price"], r["days_on_market"], r["Agent"], r['Buyer Agent'], r['Sq. Ft.'], r['address_only'], r['building_name'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    text = f'Building Name: {bold_start}{info[-1]}{bold_end}{new_line} \
    Sale Price: {bold_start}{info[0]}{bold_end}{new_line}Days on Market: \
    {bold_start}{info[1]}{bold_end}{new_line}Listing Agent: {bold_start}{info[2]} \
    {bold_end}{new_line}Buying Agent: {bold_start}{info[3]}{bold_end}{new_line}Price sq ft: \
    {bold_start}${info[4]}{bold_end}{new_line}Address: {bold_start}{info[5]}{bold_end}'
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(marker_cluster)

m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))

m

In [15]:
m.save('index.html')